In [7]:
#Import relevant libraries
import io
import itertools
import sklearn.metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorboard.plugins.hparams import api as hp

In [8]:
#Dataset encoding explaination.
#image labelling - 0=glasses/sunglasses. 1= trousers/jeans. 3= shoes

In [9]:
#Import the datasets and preprocess

data_train = np.load(r'Full Dataset/Trousers & Jeans - Male - Test.npz')
data_validation = np.load(r'Full Dataset/Trousers & Jeans - Male - Validation.npz')
data_test = np.load(r'Full Dataset/Trousers & Jeans - Male - Test.npz')


#t extract the arrays from dataset into input(images) and target(labels)
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_validation['images']
labels_val = data_validation['labels']


images_test = data_test['images']
labels_test = data_test['labels']

#Pixel-wise normalization of the training, validation and testing data
images_train = images_train/255.0
images_val  = images_val/255.0
images_test = images_test/255.0







In [10]:
#Define the hyperparameters

BATCH_SIZE = 64
EPOCHS = 15

#Define the hyperparamets to tune and the variations we want to test.
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filter_num', hp.Discrete([32, 64, 96, 128]))


METRIC_ACCURACY = 'accuracy'

#Log the hyperparameter with the file writer
with tf.summary.create_file_writer('Logs/ModelM 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM], 
        metrics= [hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )



In [11]:
def train_model(hparams,session_num):
    
    
#Create and train the model


    
    model = Sequential([
                Conv2D(32, 3, activation= 'relu', input_shape=(120,90,3)),
                MaxPooling2D(pool_size=(2,2)),
                Conv2D(64, 3, activation= 'relu'),
                MaxPooling2D(pool_size=(2,2)),
                Flatten(),
                Dense(64, activation = 'relu'),
                
                Dense(2)
    ])


#describe the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Compile the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    log_dir = f"Logs/ModelM 1/fit/run-{session_num}"

    
        
        
        
#Plot_to_image

    def plot_confusion_matrix(cm, class_names):
 
        figure = plt.figure(figsize=(6, 6))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Compute the labels from the normalized confusion matrix.
        labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    



    def plot_to_image(figure):
       
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image


        
    # log the confusion matrix as a heatmap
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')


    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        val_pred_raw = model.predict(images_val)
        val_pred = np.argmax(val_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = confusion_matrix(labels_val, val_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=["Trouser", "Jeans"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion_matrix", cm_image, step=epoch)

    # Define the per-epoch callback.
    cm_callback = LambdaCallback(on_epoch_end=log_confusion_matrix)


    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


    early_stopping = EarlyStopping(
        monitor= 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0,
        restore_best_weights = True
    )


#Train the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val, labels_val),
        verbose =2
    )

    _,accuracy = model.evaluate(images_val, labels_val)

    model.save(f"saved_model/ModelM 1/fit/run-{session_num}")


    return accuracy



In [12]:
#Log the hyperparameter to the TensorBoard



def run(log_dir, hparams,session_num):
    hyperparameter_writer = tf.summary.create_file_writer(log_dir)
    
    
    with hyperparameter_writer.as_default():
        hp.hparams(hparams) #record the values used in this trial
        accuracy = train_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        
        
        
        
        
#Train the model with the different hyperparameters
session_num = 1
for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        hparams ={
        HP_FILTER_SIZE: filter_size,
        HP_FILTER_NUM : filter_num
        }
        
        run_name = "run-%d" % session_num
        print('---Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/ModelM 1/hparam_tuning/' + run_name, hparams, session_num)
        
        session_num += 1
       
        

       
        
        
        








---Starting trial: run-1
{'filter_size': 3, 'filter_num': 32}
Epoch 1/15
8/8 [==============================] - 1s 65ms/step
4/4 - 7s - loss: 2.3746 - accuracy: 0.4720 - val_loss: 0.8666 - val_accuracy: 0.4720 - 7s/epoch - 2s/step
Epoch 2/15
8/8 [==============================] - 1s 61ms/step
4/4 - 4s - loss: 0.7639 - accuracy: 0.5120 - val_loss: 0.7007 - val_accuracy: 0.4720 - 4s/epoch - 901ms/step
Epoch 3/15
8/8 [==============================] - 1s 63ms/step
4/4 - 4s - loss: 0.6857 - accuracy: 0.5240 - val_loss: 0.6791 - val_accuracy: 0.5400 - 4s/epoch - 880ms/step
Epoch 4/15
8/8 [==============================] - 1s 64ms/step
4/4 - 4s - loss: 0.6783 - accuracy: 0.6280 - val_loss: 0.6679 - val_accuracy: 0.7280 - 4s/epoch - 915ms/step
Epoch 5/15
8/8 [==============================] - 0s 57ms/step
4/4 - 4s - loss: 0.6606 - accuracy: 0.7480 - val_loss: 0.6477 - val_accuracy: 0.6920 - 4s/epoch - 896ms/step
Epoch 6/15
8/8 [==============================] - 0s 56ms/step
4/4 - 3s - loss: 0

INFO:tensorflow:Assets written to: saved_model/ModelM 1/fit/run-1\assets


---Starting trial: run-2
{'filter_size': 3, 'filter_num': 64}
Epoch 1/15
8/8 [==============================] - 1s 65ms/step
4/4 - 6s - loss: 2.0972 - accuracy: 0.5560 - val_loss: 0.6288 - val_accuracy: 0.6800 - 6s/epoch - 1s/step
Epoch 2/15
8/8 [==============================] - 1s 61ms/step
4/4 - 4s - loss: 0.9071 - accuracy: 0.5080 - val_loss: 0.6397 - val_accuracy: 0.6520 - 4s/epoch - 896ms/step
Epoch 3/15
8/8 [==============================] - 1s 61ms/step
4/4 - 4s - loss: 0.6432 - accuracy: 0.6080 - val_loss: 0.6338 - val_accuracy: 0.7040 - 4s/epoch - 899ms/step
8/8 [==============================] - 1s 64ms/step - loss: 0.6288 - accuracy: 0.6800
INFO:tensorflow:Assets written to: saved_model/ModelM 1/fit/run-2\assets


INFO:tensorflow:Assets written to: saved_model/ModelM 1/fit/run-2\assets


---Starting trial: run-3
{'filter_size': 3, 'filter_num': 96}
Epoch 1/15
8/8 [==============================] - 1s 61ms/step
4/4 - 6s - loss: 2.3437 - accuracy: 0.5040 - val_loss: 0.9742 - val_accuracy: 0.4720 - 6s/epoch - 1s/step
Epoch 2/15
8/8 [==============================] - 1s 67ms/step
4/4 - 4s - loss: 0.8317 - accuracy: 0.5120 - val_loss: 0.7313 - val_accuracy: 0.4720 - 4s/epoch - 895ms/step
Epoch 3/15
8/8 [==============================] - 1s 66ms/step
4/4 - 4s - loss: 0.6958 - accuracy: 0.5160 - val_loss: 0.6931 - val_accuracy: 0.5240 - 4s/epoch - 921ms/step
Epoch 4/15
8/8 [==============================] - 1s 68ms/step
4/4 - 4s - loss: 0.6941 - accuracy: 0.4880 - val_loss: 0.6930 - val_accuracy: 0.5280 - 4s/epoch - 995ms/step
Epoch 5/15
8/8 [==============================] - 1s 63ms/step
4/4 - 4s - loss: 0.6932 - accuracy: 0.4880 - val_loss: 0.6930 - val_accuracy: 0.5280 - 4s/epoch - 976ms/step
Epoch 6/15
8/8 [==============================] - 1s 66ms/step
4/4 - 4s - loss: 0

...

INFO:tensorflow:Assets written to: saved_model/ModelM 1/fit/run-10\assets


---Starting trial: run-11
{'filter_size': 7, 'filter_num': 96}
Epoch 1/15
8/8 [==============================] - 1s 65ms/step
4/4 - 6s - loss: 1.2940 - accuracy: 0.5320 - val_loss: 0.8895 - val_accuracy: 0.4720 - 6s/epoch - 1s/step
Epoch 2/15
8/8 [==============================] - 1s 67ms/step
4/4 - 4s - loss: 0.7413 - accuracy: 0.5120 - val_loss: 0.6852 - val_accuracy: 0.5280 - 4s/epoch - 946ms/step
Epoch 3/15
8/8 [==============================] - 1s 64ms/step
4/4 - 4s - loss: 0.6875 - accuracy: 0.5040 - val_loss: 0.6624 - val_accuracy: 0.7360 - 4s/epoch - 975ms/step
Epoch 4/15
8/8 [==============================] - 1s 65ms/step
4/4 - 4s - loss: 0.6617 - accuracy: 0.6160 - val_loss: 0.5999 - val_accuracy: 0.6800 - 4s/epoch - 891ms/step
Epoch 5/15
8/8 [==============================] - 1s 68ms/step
4/4 - 4s - loss: 0.6071 - accuracy: 0.6840 - val_loss: 0.5673 - val_accuracy: 0.7040 - 4s/epoch - 903ms/step
Epoch 6/15
8/8 [==============================] - 1s 65ms/step
4/4 - 4s - loss: 

INFO:tensorflow:Assets written to: saved_model/ModelM 1/fit/run-11\assets


---Starting trial: run-12
{'filter_size': 7, 'filter_num': 128}
Epoch 1/15
8/8 [==============================] - 1s 66ms/step
4/4 - 6s - loss: 1.0362 - accuracy: 0.4960 - val_loss: 0.6603 - val_accuracy: 0.5280 - 6s/epoch - 1s/step
Epoch 2/15
8/8 [==============================] - 1s 64ms/step
4/4 - 4s - loss: 0.6852 - accuracy: 0.4960 - val_loss: 0.6936 - val_accuracy: 0.4720 - 4s/epoch - 906ms/step
Epoch 3/15
8/8 [==============================] - 1s 66ms/step
4/4 - 4s - loss: 0.6643 - accuracy: 0.6040 - val_loss: 0.6266 - val_accuracy: 0.7280 - 4s/epoch - 913ms/step
Epoch 4/15
8/8 [==============================] - 1s 62ms/step
4/4 - 4s - loss: 0.6253 - accuracy: 0.7120 - val_loss: 0.5786 - val_accuracy: 0.7200 - 4s/epoch - 877ms/step
Epoch 5/15
8/8 [==============================] - 1s 65ms/step
4/4 - 4s - loss: 0.5660 - accuracy: 0.7560 - val_loss: 0.5336 - val_accuracy: 0.7360 - 4s/epoch - 957ms/step
Epoch 6/15
8/8 [==============================] - 1s 65ms/step
4/4 - 4s - loss:

INFO:tensorflow:Assets written to: saved_model/ModelM 1/fit/run-12\assets


In [13]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/ModelM 1/hparam_tuning'


In [14]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/ModelM 1/fit'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
